<a href="https://colab.research.google.com/github/saided/pandas_using_python/blob/master/GRP_Creation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import requests
import xlrd
import openpyxl
from collections import ChainMap
from requests import Timeout

In [4]:
def root_group():
    url = "https://api.cloud.jiosurveillance.com/auth/oauth/token"
    username = input("Enter account ID")
    password = input("Enter account password")

    payload = 'client_secret=8b078e3496893ae3b9d7ccb83a6befdb&client_id=firmware-upgrade-job&grant_type=password&username={}&password={}'.format(
    username, password)
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
            }
    try:
        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        #print(response.text.encode('utf8'))
        #print(response.json())
    except: Timeout

    r = response.json()
    access_token = r['access_token']
    token = str(access_token)
    #print(token)
    own_id = r['owner_id']

    url = "https://api.cloud.jiosurveillance.com/folders?op=FIND"

    payload = "{\n\t\"user\" : \"%s\"\n}" % (own_id)
    headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % (token)
    }
    response = requests.request("POST", url, headers=headers, data=payload, timeout=6)

    own_response = response.json()
    print(own_response)
    b = own_response['result']
    c = b['items']
    #print(c[0])
    #print(len(c))
    #print('Total no.of existing groups in the account :',len(c)-1)
    mast_grp_id = c[0]['id']
    #print(mast_grp_id,type(mast_grp_id))
    if len(c)==0:
      print('something went wrong')
    return(mast_grp_id,own_id,token)

print(root_group())  

Enter account IDrr.test4@gmail.com
Enter account passwordJioadmin@123
{'success': True, 'result': {'items': [{'id': '801-Z6hTHMk9Ii', 'name': '__root__', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': True, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}]}}
('801-Z6hTHMk9Ii', 801000012219, '801-Ub8289477-c584-4a48-8d18-878c6ee72445')


In [47]:
is_root_grp = list(root_group())
print(len(is_root_grp))
mast_grp_id = is_root_grp[0]
own_id = is_root_grp[1]
token = is_root_grp[2]
#print(token,type(token))

Enter account IDrr.test4@gmail.com
Enter account passwordJioadmin@123
{'success': True, 'result': {'items': [{'id': '801-Z6hTHMk9Ii', 'name': '__root__', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': True, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}]}}
3


In [48]:

def create_group(cellvalue,grp_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + grp_id + '\",\n\t\"name\" : \"' + cellvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)




In [49]:
North=0
theFile = openpyxl.load_workbook('/content/Tree structure.xlsx')
allSheetNames = theFile.sheetnames
#print(allSheetNames)
print("All sheet names {} " .format(allSheetNames))
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  print('Total no of rows',sh.nrows)
  print('Total no of columns',sh.ncols)
  zone = []
  zone_folder = []
  northList = []
  northtownList = []
  jplist = []
  iplist = []
  state_folder = []
  town_folder = []
  jp_folder = []
  ip_folder = []
  zone_id = []
  state_id = []
  town_id = []
  jp_id = []
  ip_id = []
  i= 0
  j=0
  k = 0
  l=0
  m = 0
  for row in range(1, sh.nrows):
    for col in range(sh.ncols):
      mycell = sh.cell(row,col).value
      if (len(is_root_grp) !=0 and col==6):
        if mycell not in zone:
             zone.append(mycell)
             print(zone)
             zone_folder.append(create_group(mycell,mast_grp_id,token))
             zone_id.append(zone_folder[i]['result']['id'])
             i+=1
        else:
           break
    
    for col in range(1, 7):
      mycell = sh.cell(row,col).value
      for x in range(len(zone)):
          if mycell==zone[x]:
            print(x,mycell)
            print(row,col)
            statecell = sh.cell(row, col+1).value
        #statecell.value = str(statecell.value)
            if statecell.upper() in [name.upper() for name in northList]:
              break
            else:
              northList.append(statecell)
              print("NorthList contains following list" + str(northList))
              state_folder.append(create_group(statecell,zone_id[x],token))
              print(j)
              state_id.append(state_folder[j]['result']['id'])
              j+=1
    for col in range(0, 9):
      mycell = sh.cell(row,col).value
      for x in range(len(northList)):
          if mycell == northList[x]:
            town = sh.cell(row, col+1).value
            if town.lower() in [name.lower() for name in northtownList]:
              break
            else:
              northtownList.append(town)
              town_folder.append(create_group(town,state_id[x],token))
              print(k)
              town_id.append(town_folder[k]['result']['id'])
              k+=1
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      for x in range(len(northtownList)):
          if mycell ==northtownList[x] :
            jp = sh.cell(row,col+2).value
            if jp.lower() in [name.lower() for name in jplist]:
              break
            else:
              jplist.append(jp)
              jp_folder.append(create_group(jp,town_id[x],token))
              jp_id.append(jp_folder[l]['result']['id'])
              l+=1
   
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      for x in range(len(jplist)):
          if mycell ==jplist[x] :
            ip = sh.cell(row,col-9).value
            if ip.lower() in [name.lower() for name in iplist]:
              break
            else:
              iplist.append(ip)
              ip_folder.append(create_group(ip,jp_id[x],token))
              ip_id.append(ip_folder[m]['result']['id'])
              m+=1
            

    
    
    



All sheet names ['Sheet1'] 
Total no of rows 192
Total no of columns 12
['WEST']
{
	"parent_folder" : "801-Z6hTHMk9Ii",
	"name" : "WEST"
}
0 WEST
1 6
NorthList contains following list['MAHARASHTRA']
{
	"parent_folder" : "801-s51fKV6Dv0",
	"name" : "MAHARASHTRA"
}
0
{
	"parent_folder" : "801-oaVVruq0KS",
	"name" : "Ratnagiri"
}
0
{
	"parent_folder" : "801-kjkPMl67yX",
	"name" : "I-MH-KUDL-ENB-9003"
}
{
	"parent_folder" : "801-N3ak629p6G",
	"name" : "10.208.230.141"
}
['WEST', 'EAST']
{
	"parent_folder" : "801-Z6hTHMk9Ii",
	"name" : "EAST"
}
1 EAST
2 6
NorthList contains following list['MAHARASHTRA', 'ORISSA']
{
	"parent_folder" : "801-y8Pk5Ms3ej",
	"name" : "ORISSA"
}
1
{
	"parent_folder" : "801-e2H9Fxb5Ss",
	"name" : "Bhubaneshwar"
}
1
{
	"parent_folder" : "801-vz7F1mBbxk",
	"name" : "I-OR-CNDK-ENB-6000"
}
{
	"parent_folder" : "801-s33ON1NGrG",
	"name" : "10.216.50.85"
}
1 EAST
3 6
{
	"parent_folder" : "801-e2H9Fxb5Ss",
	"name" : "Beti"
}
2
{
	"parent_folder" : "801-I0GbPSG4J0",
	"name

In [32]:
#print(zone_folder)
#zone_id = zone_folder[0]['result']['id']
print(zone_id)
print(state_folder[2])
print(northList)
print(len(state_id))
print(len(town_id))

['801-JYoAiuXOQO', '801-Kma9oZDKvW', '801-XyeXGF2Ou9', '801-pngyzrsLoX']
{'success': True, 'result': {'id': '801-QAX2zBBKBD', 'name': 'MADHYA PRADESH', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}
['MAHARASHTRA', 'ORISSA', 'MADHYA PRADESH', 'GUJARAT', 'JAMMU KASHMIR', 'TAMIL NADU', 'ASSAM', 'HARYANA', 'RAJASTHAN', 'UP WEST', 'UTTARAKHAND', 'ANDHRA PRADESH', 'KERALA', 'UP EAST1', 'JHARKHAND']
15
62


In [42]:
print(town_folder[26])
print(jplist)

{'success': True, 'result': {'id': '801-XVYCibkPPz', 'name': 'Bhadrak', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}
['I-MH-KUDL-ENB-9003', 'I-OR-CNDK-ENB-6000', 'I-OR-BETI-ENB-6004', 'I-OR-BLSR-ENB-9005', 'I-OR-RRKL-ENB-6010', 'I-OR-KTPD-ENB-9002', 'I-MP-JBLP-ENB-0002', 'I-GJ-BVLA-ENB-9018', 'I-OR-RRKL-ENB-6015', 'I-OR-RRKL-ENB-6032', 'I-OR-BRRI-ENB-9001', 'I-OR-CNDP-ENB-6002', 'I-OR-JGTP-ENB-6003', 'I-OR-PRDP-ENB-6002', 'I-OR-CCKK-ENB-6030', 'I-OR-GOLT-ENB-9004', 'I-JK-RESI-ENB-0005', 'I-TN-LLGD-ENB-9009', 'I-MP-SHOR-ENB-9011', 'I-AS-RMKS-ENB-9011', 'I-OR-RRKL-ENB-6034', 'I-MP-RTLM-AG1-4400', 'I-MP-GUNA-ENB-9010', 'I-AS-TNSK-ENB-6002', 'I-OR-SNTL-ENB-9005', 'I-OR-RGNP-ENB-9021', 'I-HR-GHAN-ENB-9019', 'I-RJ-PNDW-ENB-9015', 'I-OR-KNDP-ENB-9003', 'I-OR-RGNP-ENB-6005', 'I-OR-SUKN-ENB-9004', 'I-RJ-PNDW-ENB-

In [ ]:

def create_substate(statevalue,zone_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + zone_id + '\",\n\t\"name\" : \"' + statevalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)




In [ ]:
#print(n)
#zone_id = n['result']['id']
#print(zone_id)
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  northList = []
  state_folder = []
  for row in range(1, sh.nrows):
    for col in range(1, 7):
      mycell = sh.cell(row,col).value
      if mycell=='NORTH':
        statecell = sh.cell(row, col-6).value
        #statecell.value = str(statecell.value)
        if statecell.lower() in [name.lower() for name in northList]:
          break
        else:
          northList.append(statecell)
          print("NorthList contains following list" + str(northList))
          state_folder.append(create_substate(statecell,zone_id,token))
          #print("Received folder id for NORTH is" + folder_id[0])
          #check_state_present = check_state_created(cell_content, folder_id)
          #create_folder(folder_id[0], stateCell.value, token)

NorthList contains following list['Jammu Kashmir']
{
	"parent_folder" : "801-ANTNDx6mEs",
	"name" : "Jammu Kashmir"
}
NorthList contains following list['Jammu Kashmir', 'Haryana']
{
	"parent_folder" : "801-ANTNDx6mEs",
	"name" : "Haryana"
}
NorthList contains following list['Jammu Kashmir', 'Haryana', 'Rajasthan']
{
	"parent_folder" : "801-ANTNDx6mEs",
	"name" : "Rajasthan"
}
NorthList contains following list['Jammu Kashmir', 'Haryana', 'Rajasthan', 'UP West']
{
	"parent_folder" : "801-ANTNDx6mEs",
	"name" : "UP West"
}
NorthList contains following list['Jammu Kashmir', 'Haryana', 'Rajasthan', 'UP West', 'Uttarakhand']
{
	"parent_folder" : "801-ANTNDx6mEs",
	"name" : "Uttarakhand"
}
NorthList contains following list['Jammu Kashmir', 'Haryana', 'Rajasthan', 'UP West', 'Uttarakhand', 'UP EAST']
{
	"parent_folder" : "801-ANTNDx6mEs",
	"name" : "UP EAST"
}


In [ ]:
print(len(state_folder))
state_id = []
for x in range(len(state_folder)):
  state_id.append(state_folder[x]['result']['id'])
print(state_id)

6
['801-Eaf8C6EaiH', '801-N9Gt1g4bvB', '801-z6FtD3n7nh', '801-Q309qEtgZS', '801-UPVc3ztZMs', '801-EnwWQIfLvf']


In [ ]:
def create_town(townvalue,state_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + state_id + '\",\n\t\"name\" : \"' + townvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)


In [ ]:
#print(state_folder)
#state_id = state_folder['result']['id']
#print(state_id)
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  northtownList = []
  town_folder=[]
  i=0
  for row in range(1, sh.nrows):
    for col in range(0,9):
      mycell = sh.cell(row,col).value
      for x in range(len(northList)):
        if mycell == northList[x]:
          town = sh.cell(row, col+8).value
        #statecell.value = str(statecell.value)
          if town.lower() in [name.lower() for name in northtownList]:
            #break
          #else:
          northtownList.append(town)
            #print("NorthtownList contains following list" + str(northtownList))
          town_folder.append(create_town(town,state_id[x],token))
          #print("Received folder id for NORTH is" + folder_id[0])
          #check_state_present = check_state_created(cell_content, folder_id)
          #create_folder(folder_id[0], stateCell.value, token)


In [ ]:
#print(town_folder[1])
town_id = []
print(len(town_folder))
for x in range(len(town_folder)):
  try:
    #print(x,town_folder[x]['result']['id'])
    town_id.append(town_folder[x]['result']['id'])
  except KeyError as error:
    #print(x,error)
    town_id.append("801-z6FtD3n7nh")
  #town_id.append(town_folder[x]['result']['id'])
  #print(town_folder[x]['success']['id'])
#print(type(town_id[14]))
#print(len(town_id))
print(town_id)




146
['801-WpN0dcS7Lw', '801-irIYv3JHQz', '801-DGbnkcrk9I', '801-YiRkzpcd4U', '801-Wqiz9gUA0g', '801-Fv7Dd5kuPq', '801-WBGvve1Wlx', '801-X02y1t5bLY', '801-oxBq0CZgho', '801-L4JvJV9pr9', '801-YBB1CfsyxZ', '801-SmxshkxcRi', '801-Bk3J9iNjHk', '801-dvkFb0lkP7', '801-Dj3JzZQ9p0', '801-96u2rzvRiX', '801-WMQ4gTC1m7', '801-4bLngscq6V', '801-fxDCiWUlev', '801-OuTj8E97ny', '801-P5DfWksNei', '801-gmhF0IqHU4', '801-Mrlv7srJ5z', '801-U7B25cgGGq', '801-6z1iHaqkFS', '801-WOcgzQCcdz', '801-DSo1u1DUNp', '801-v50d1ZD9hY', '801-06BXiWqNMS', '801-EL9YEQkZ4B', '801-GUNSRUZnzi', '801-9a82itqZnE', '801-8xC4TosrrB', '801-9u7D1CqB39', '801-T04jkhqxNP', '801-XJNQSzxdl6', '801-MuJLqR9Se6', '801-D6xhheg6Po', '801-vhtX5YCiUv', '801-MRgqx0Y7X2', '801-RPbD2RGCdJ', '801-U6EarznETg', '801-OTSFkUXzPx', '801-f5AjrqXL3f', '801-s9ccVqUzHf', '801-rjynDEj6Km', '801-SnGShME40O', '801-H5ZQnYJVUR', '801-dTjxRyTRmY', '801-F48l1uJvXl', '801-dHoLiG4QD9', '801-TDh4wuof8y', '801-GxSjdwo4g7', '801-rRQboksQ9q', '801-7vlKD4bYyl', '801-

In [ ]:
def create_jp(jpvalue,town_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + town_id + '\",\n\t\"name\" : \"' + jpvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)

In [ ]:
#jplist = []
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  jplist = []
  jp_folder = []
  i=0
  #print(northtownList)
  for row in range(1, sh.nrows):
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      #print(mycell)
      for x in northtownList:
        #print(x)
        if mycell ==x :
          #print(mycell,x)
          jp = sh.cell(row,col+2).value
          #print('jp',jp)
          if jp.lower() in [name.lower() for name in jplist]:
            break
          else:
            jplist.append(jp)
            jp_folder.append(create_jp(jp,town_id[i],token))
            i+=1

In [ ]:
jp_idlist = []
for x in range(len(jp_folder)):
   jp_idlist.append(jp_folder[x]['result']['id'])
   #print(jp_folder[x]['result']['id'])
   #i=0
print(len(jp_idlist))
#for x in range(len(jp_idlist)):
    #print(jp_idlist[1])
    #i+=1

146


In [ ]:
def create_city(cityvalue,jp_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + jp_id + '\",\n\t\"name\" : \"' + cityvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)

In [ ]:
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  jpcity = []
  city_folder=[]
  i=0
  #print(northtownList)
  for row in range(1, sh.nrows):
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      #print(mycell)
      for x in jplist:
        #print(x)
        if mycell ==x :
          #print(mycell,x)
          city = sh.cell(row,col-9).value
          #print('jp',jp)
          if city.lower() in [name.lower() for name in jpcity]:
            break
          else:
            jpcity.append(city)
            #for x in jp_idlist:
            city_folder.append(create_city(city,jp_idlist[i],token))
            i+=1
            
#print(jpcity)

{
	"parent_folder" : "801-zFt546JF9n",
	"name" : "10.213.234.197"
}
{
	"parent_folder" : "801-kEz6kNA3w9",
	"name" : "10.213.37.213"
}
{
	"parent_folder" : "801-HjRFWOheh4",
	"name" : "10.211.3.45"
}
{
	"parent_folder" : "801-vuTUiOLoiK",
	"name" : "10.211.159.69"
}
{
	"parent_folder" : "801-rLgNVDCdhv",
	"name" : "10.215.165.77"
}
{
	"parent_folder" : "801-cJNzKPeIdZ",
	"name" : "10.215.136.69"
}
{
	"parent_folder" : "801-x8ItYV4Rmh",
	"name" : "10.215.135.253"
}
{
	"parent_folder" : "801-3332WlioZY",
	"name" : "10.215.136.61"
}
{
	"parent_folder" : "801-hdE2dXjVHo",
	"name" : "10.215.139.45"
}
{
	"parent_folder" : "801-0UI2Vm6Hs1",
	"name" : "10.215.222.53"
}
{
	"parent_folder" : "801-sNtMJa4UID",
	"name" : "10.214.249.173"
}
{
	"parent_folder" : "801-J6W20eLnFA",
	"name" : "10.214.243.133"
}
{
	"parent_folder" : "801-hwRMGsH4Ph",
	"name" : "10.214.243.149"
}
{
	"parent_folder" : "801-8BK5EtWQJr",
	"name" : "10.214.10.109"
}
{
	"parent_folder" : "801-YUUPgFp0y1",
	"name" : "10.214.24

In [ ]:
#print(city_folder)
city_idlist = []
for x in range(len(city_folder)):
   city_idlist.append(city_folder[x]['result']['id'])
   #print(jp_folder[x]['result']['id'])
print(city_idlist)

['801-pVgM2xqn31', '801-GTn51FbSHL', '801-9JLDN1MQ4H', '801-hklvXQG0ek', '801-djDTEMx84M', '801-uR3EDAWND0', '801-pk3OU2lAob', '801-S36xkewc4b', '801-mTyfTEpRKe', '801-cHUpLCqy93', '801-63JVmu0tLi', '801-OBdOUBqt3h', '801-MqqqxMEwB8', '801-lK8DZmOlSQ', '801-D8KUqxDvtI', '801-TUFr8ggRdH', '801-FAT5ficThH', '801-wfg65wgKbx', '801-E12nvir4Oc', '801-wAD5YhECFg', '801-HRcO4cg1QM', '801-BZWhMXDsrB', '801-msdthMGjyj', '801-FtXVee7k9j', '801-2uxvoLujhH', '801-ZjpEVFm91w', '801-WJKR5JHdnL', '801-ezu5xas5v6', '801-xXzMOgUhxk', '801-t1M8wQRRXo', '801-g1JyhDj281', '801-BRe4Mo6i2i', '801-9Uo3onbh3T', '801-zAX1aZQbik', '801-qC6QKf6Qxx', '801-0a6xtUFGsi', '801-CJoHqeaQao', '801-N44644HHcL', '801-ldhECz0ruq', '801-aRaqOkAbfU', '801-LFGoegeDkS', '801-osQi3bRLuJ', '801-RBoH7x0oej', '801-7iWAvRblb5', '801-zlfuzmTq1r', '801-fwcsW9MuDq', '801-AuuF025fgq', '801-Ca7o7OJrd2', '801-eEhJjKPbxP', '801-VFvolmFblg', '801-BxXj8eoAOI', '801-DVdLfM17kd', '801-pKgiO5bjTm', '801-X7yVpQfdGX', '801-YScK09ZtD8', '801-rpho

In [ ]:
def create_mail(mailvalue,city_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + city_id + '\",\n\t\"name\" : \"' + mailvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)

In [ ]:
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  jpmail = []
  i=0
  mail_folder = []
  #print(northtownList)
  for row in range(1, sh.nrows):
    for col in range(1, 12):
      mycell = sh.cell(row,col).value
      #print(mycell)
      for x in jpcity:
        #print(x)
        if mycell ==x :
          #print(mycell,x)
          mail = sh.cell(row,col+10).value
          #print('jp',jp)
          if mail.lower() in [name.lower() for name in jpmail]:
            break
          else:
            jpmail.append(mail)
            mail_folder.append(create_mail(mail,city_idlist[i],token))
            i+=1


{
	"parent_folder" : "801-pVgM2xqn31",
	"name" : "INJKRESIRESITW0004ENBCAM001"
}
{
	"parent_folder" : "801-GTn51FbSHL",
	"name" : "INHRGHANXXXXTW0001ENBCAM001"
}
{
	"parent_folder" : "801-9JLDN1MQ4H",
	"name" : "INRJPNDWXXXXXXXXXXENBCAM001"
}
{
	"parent_folder" : "801-hklvXQG0ek",
	"name" : "INRJPNDWXXXXTW0013ENBCAM001"
}
{
	"parent_folder" : "801-djDTEMx84M",
	"name" : "INUWMWNAXXXXTW0003ENBCAM001"
}
{
	"parent_folder" : "801-uR3EDAWND0",
	"name" : "INUWBALCBMHITW0001ENBCAM001"
}
{
	"parent_folder" : "801-pk3OU2lAob",
	"name" : "INUWKHTLXXXXTW0004ENBCAM001"
}
{
	"parent_folder" : "801-S36xkewc4b",
	"name" : "INUWMZGRXXXXTW0005ENBCAM001"
}
{
	"parent_folder" : "801-mTyfTEpRKe",
	"name" : "INUWKHTLXXXXTW0003ENBCAM001"
}
{
	"parent_folder" : "801-cHUpLCqy93",
	"name" : "INUWPRQZPRQZTW0001ENBCAM001"
}
{
	"parent_folder" : "801-63JVmu0tLi",
	"name" : "INUWMWNAXXXXTW0001ENBCAM001"
}
{
	"parent_folder" : "801-OBdOUBqt3h",
	"name" : "INUWDBNDXXXXTW0001ENBCAM001"
}
{
	"parent_folder" : "801-Mq

In [ ]:
print(mail_folder)

[{'success': True, 'result': {'id': '801-sbFGXSXSob', 'name': 'INJKRESIRESITW0004ENBCAM001', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-QdbYEB3mdG', 'name': 'INHRGHANXXXXTW0001ENBCAM001', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-suahWKUYYr', 'name': 'INRJPNDWXXXXXXXXXXENBCAM001', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'